In [1]:
import torch
import torchvision.models as models
import numpy as np
import json
import os
import cv2
import collections

#config
device = torch.device('cuda:0')
batch_size=32
lr=0.01
epochs=10

In [2]:
#input data
label=[]
img=[]
#f1=open("./week10_dataset/0_annotation_train.txt",'r', encoding='UTF-8')
#for line in f1:
#    dic = json.loads(line)
#   label.append(dic["annotation"][0]["name"])
#   img.append('./week10_dataset'+dic["source"].split("image")[-1])

f1=open("./week10_dataset_bak/train_set_0.txt",'r', encoding='UTF-8')
for line in f1:
    dic = line.split(',')
    label.append(int(dic[1]))
    img.append('./week10_dataset_bak/'+dic[0])
print(label)
print(img)

[0, 1, 0, 2, 3, 4, 4, 5, 3, 4, 1, 7, 0, 0, 0, 0, 5, 5, 5, 8, 4, 2, 2, 5, 0, 0, 8, 0, 1, 6, 2, 0, 0, 5, 4, 5, 7, 4, 4, 4, 4, 5, 4, 0, 4, 0, 2, 0, 9, 5, 8, 2, 2, 2, 8, 0, 9, 8, 2, 4, 0, 8, 0, 3, 3, 8, 0, 0, 2, 4, 2, 4, 0, 5, 4, 8, 0, 8, 4, 2, 0, 6, 0, 4, 2, 8, 8, 13, 0, 5, 1, 4, 10, 2, 5, 2, 2, 2, 5, 8, 2, 4, 8, 11, 0, 8, 10, 0, 3, 0, 8, 0, 5, 4, 3, 4, 10, 7, 2, 0, 0, 6, 0, 0, 2, 0, 5, 4, 4, 4, 4, 10, 2, 0, 4, 5, 5, 9, 4, 5, 8, 6, 2, 4, 2, 8, 5, 0, 2, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 1

In [4]:


def train_data_generator(imgs, labels, batch_size=32, img_size=(224,224)):
    batch_index = np.arange(0, len(imgs))
    img_out=[]
    label_out=[]
    while True:
        np.random.shuffle(batch_index)
        for i in batch_index:
            if os.path.exists(imgs[i]):
                img =cv2.imdecode(np.fromfile(imgs[i],dtype=np.uint8),1)
                label_out.append(labels[i])
                train_img= cv2.resize(img, (img_size[0], img_size[1]), interpolation=cv2.INTER_LINEAR)
                img_out.append(train_img)
                if len(img_out)>=batch_size:
                    img_out = np.array(img_out, dtype=np.float32)
                    img_out = img_out[:, :, :, ::-1]
                    img_out=torch.from_numpy(np.array(img_out))
                    img_out = img_out.permute(0, 3, 1, 2).float() / (255.0 / 2) - 1
                    label_out = np.array(label_out, dtype=np.int64)
                    label_out=torch.from_numpy(np.array(label_out))
                    label_out = label_out.long()
                    yield img_out, label_out
                    img_out, label_out=[], []
            else:
                print(imgs[i], 'not exist')
data_generator=train_data_generator(img, label, batch_size=batch_size, img_size=(224,224))


In [6]:
#model
model = models.resnet18(pretrained=True).to(device)
fc_features = model.fc.in_features
model.fc = torch.nn.Linear(fc_features, 14).to(device)
optimizer = torch.optim.Adam([model.fc.weight,model.fc.bias], lr=lr)
criterion = torch.nn.CrossEntropyLoss()


In [7]:
print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
#training
model.train()
epoch_size=int(len(img)/batch_size)
for epoch in range(epochs):
    model.train()
    epoch_loss=0.0
    correct = 0
    total = 0
    for iter in range(1, epoch_size + 1):
        imgs, labels = next(data_generator)
        imgs = imgs.to(device)
        labels = labels.to(device)
        predicts = model(imgs)
        optimizer.zero_grad()
        loss = criterion(predicts, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predicted = predicts.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        print( 'TRAIN:[epoch-%d , iter-%d]  Loss: %.3f | Acc: %.3f%% (%d/%d)' % (epoch, iter,epoch_loss / iter, 100. * correct / total, correct, total))



TRAIN:[epoch-0 , iter-1]  Loss: 2.872 | Acc: 9.375% (3/32)
TRAIN:[epoch-0 , iter-2]  Loss: 4.107 | Acc: 7.812% (5/64)
TRAIN:[epoch-0 , iter-3]  Loss: 4.338 | Acc: 9.375% (9/96)
TRAIN:[epoch-0 , iter-4]  Loss: 4.619 | Acc: 7.812% (10/128)
TRAIN:[epoch-0 , iter-5]  Loss: 4.354 | Acc: 9.375% (15/160)
TRAIN:[epoch-0 , iter-6]  Loss: 4.229 | Acc: 10.417% (20/192)
TRAIN:[epoch-0 , iter-7]  Loss: 4.101 | Acc: 11.607% (26/224)
TRAIN:[epoch-0 , iter-8]  Loss: 3.949 | Acc: 12.109% (31/256)
TRAIN:[epoch-0 , iter-9]  Loss: 3.793 | Acc: 14.583% (42/288)
TRAIN:[epoch-0 , iter-10]  Loss: 3.836 | Acc: 15.312% (49/320)
TRAIN:[epoch-0 , iter-11]  Loss: 3.952 | Acc: 16.477% (58/352)
TRAIN:[epoch-1 , iter-1]  Loss: 3.457 | Acc: 28.125% (9/32)
TRAIN:[epoch-1 , iter-2]  Loss: 3.118 | Acc: 32.812% (21/64)
TRAIN:[epoch-1 , iter-3]  Loss: 2.955 | Acc: 37.500% (36/96)
TRAIN:[epoch-1 , iter-4]  Loss: 2.790 | Acc: 38.281% (49/128)
TRAIN:[epoch-1 , iter-5]  Loss: 2.573 | Acc: 40.000% (64/160)
TRAIN:[epoch-1 , iter

In [9]:
test_label=[]
test_img=[]
f1=open("./week10_dataset_bak/test_set_0.txt",'r', encoding='UTF-8')
for line in f1:
    dic = line.split(',')
    test_label.append(int(dic[1]))
    test_img.append('./week10_dataset_bak/'+dic[0])
print(test_label)
print(test_img)

test_data_generator=train_data_generator(test_img, test_label, batch_size=len(test_img), img_size=(224,224))

[5, 8, 8, 9, 2, 8, 4, 2, 12, 1, 8, 0, 1, 0, 2, 13, 1, 5, 0, 9, 10, 4, 0, 8, 2, 4, 4, 5, 2, 5]
['./week10_dataset_bak/zhangyu/微信图片_202007150229589.jpg', './week10_dataset_bak/guozhiwei/7.jpg', './week10_dataset_bak/chengsiwei/矿泉水瓶1.jpg', './week10_dataset_bak/361658066/微信图片_20200715201859.jpg', './week10_dataset_bak/zhaomingming/WechatIMG379.png', './week10_dataset_bak/boen/怡宝2.jpg', './week10_dataset_bak/zhaomingming/WechatIMG402.png', './week10_dataset_bak/xiejingjing/week10_27_5.jpg', './week10_dataset_bak/shishengwei/4.jpg', './week10_dataset_bak/361658066/微信图片_202007152019001.jpg', './week10_dataset_bak/dldleo/6.jpg', './week10_dataset_bak/henryFung/8.jpg', './week10_dataset_bak/guozhiwei/6.jpg', './week10_dataset_bak/chengsiwei/矿泉水瓶9.jpg', './week10_dataset_bak/zhaomingming/WechatIMG382.png', './week10_dataset_bak/kuzhuaiwushen/7.jpg', './week10_dataset_bak/kuzhuaiwushen/10.jpg', './week10_dataset_bak/zhaomingming/WechatIMG372.png', './week10_dataset_bak/skypower/dabe0ea999ecd76c6

In [13]:
print(len(test_img))
model.eval()
epochs=1
for epoch in range(epochs):
    epoch_loss=0.0
    correct = 0
    total = 0
    for iter in range(1, 2):
        imgs, labels = next(test_data_generator)
        imgs = imgs.to(device)
        labels = labels.to(device)
        predicts = model(imgs)
        _, predicted = predicts.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        print( 'Test:[epoch-%d , iter-%d]  | Acc: %.3f%% (%d/%d)' % (epoch, iter, 100. * correct / total, correct, total))



30
Test:[epoch-0 , iter-1]  | Acc: 86.667% (26/30)
